In [1]:
# Need to scale problem better
# Need to include the cost function

In [2]:
%reset # clears variables in workspace
#delta = 10


# import libraries
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition
import sympy as sym
import numpy as np
from IPython.display import display
from sympy import pprint

# import EOM
import dill
dill.load_session('simplified_point_mass.db')

func_map = {'sin':sin, 'cos':cos} 

sym_list = [g]+\
           [masses[base] for base in bases] +\
           [x_body]+[y_body] +\
           [dx_body]+[dy_body] +\
           [ddx_body]+[ddy_body] +\
           [F_ax['x']] +\
           [GRF['x']] + [GRF['y']]

lamb_EOMs = {}

DOFs = ['x_body'] + ['y_body'] 


for dof_i,dof in enumerate(DOFs):
    lamb_EOMs.update({dof: sym.lambdify(sym_list,EOMs[dof_i],modules = [func_map])})

    
# PARAMETERS:
masses_ = {'body': 5756.78/1000}

total_mass = 0.0
for mass in masses_:
    total_mass += masses_[mass]
total_mass

WDOFs = ['X','Y'] 
signs = ['-ve','+ve'] 
bases = ['body']
ground_constraints = ['contact','dx_exclusive','slip_+ve','slip_-ve']       

def get_var_list_zeros(m,n):
    var_list = [m.g] +\
               [m.masses[base] for base in bases] +\
               [m.q[n,dof] for dof in DOFs] +\
               [m.dq[n,dof] for dof in DOFs] +\
               [m.ddq[n,dof] for dof in DOFs] +\
               [0]+\
               [0] + [0]
    return var_list

def get_var_list_values(m,n):
    
    BW = 1   
        
    F_ax_value = BW*m.F_ax[n]
    GRF_X_value = BW*m.GRF_x[n]
    GRF_Y_value = BW*m.GRF_y[n]

    var_list = [m.g] +\
               [m.masses[base] for base in bases] +\
               [m.q[n,dof] for dof in DOFs] +\
               [m.dq[n,dof] for dof in DOFs] +\
               [m.ddq[n,dof] for dof in DOFs] +\
               [F_ax_value] + \
               [GRF_X_value] + [GRF_Y_value]
    return var_list



Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Don't know how to reset  #, please run `%reset?` for details
Don't know how to reset  clears, please run `%reset?` for details
Don't know how to reset  variables, please run `%reset?` for details
Flushing input history
Don't know how to reset  workspace, please run `%reset?` for details


In [3]:
tests = {0:"HDT",1:"MT"}


test = tests[1]
h_lower_bound = 0.5
h_higher_bound = 1.0

if test == "HDT":
    N = 100
    TT = 0.3
else:
    N = 100
    TT = 1.5

In [4]:
m = ConcreteModel('Biped_wFW')

#SETS-------------------------
m.N = RangeSet(N)
m.ground_constraints = Set(initialize = ground_constraints)
m.bases = Set(initialize = bases)
m.DOFs = Set(initialize = DOFs)
m.WDOFs = Set(initialize = WDOFs)
m.signs = Set(initialize = signs)


#Params--------------------------
m.g = Param(initialize = 9.81)

# m.masses = Param(m.links, initialize = masses)
m.masses = Param(m.bases, initialize = masses_)

#VARIABLES----------------------
#gen coordinates
m.q = Var(m.N,m.DOFs)
m.dq = Var(m.N,m.DOFs)
m.ddq = Var(m.N,m.DOFs)

In [5]:
''' CONSTRAINTS  - Integration'''
# variable timestep
hm = TT/N # master timestep
m.h = Var(m.N, bounds = (h_lower_bound,h_higher_bound)) #m.h = Var(m.N, bounds = (0.5,1.5))

def BwEuler_p(m,n,dof):
    if n > 1:
        return m.q[n,dof] == m.q[n-1,dof] + hm*m.h[n]*m.dq[n,dof]
    else:
        return Constraint.Skip
m.integrate_p = Constraint(m.N, m.DOFs, rule = BwEuler_p)

def BwEuler_v(m,n,dof):
    if n > 1:
        return m.dq[n,dof] == m.dq[n-1,dof] + hm*m.h[n]*m.ddq[n,dof]
    else:
        return Constraint.Skip 
m.integrate_v = Constraint(m.N, m.DOFs, rule = BwEuler_v)

In [6]:
# GROUND MODEL -----------------------------------------------------------------------------------------------------------

# get foot position and velocity ----------------------------------------------------------------------------------------
m.foot_position_y = Var(m.N)
for n in range(1,N+1):
    m.foot_position_y[n].setlb(0.0)
        
lamb_foot_position_y = sym.lambdify(sym_list,foot_position[1],modules = [func_map])
def def_foot_position(m,n):
    #if n == 1: return Constraint.Skip
    var_list = get_var_list_zeros(m,n)
    return m.foot_position_y[n] == lamb_foot_position_y(*var_list)
m.def_foot_position = Constraint(m.N, rule = def_foot_position)

m.foot_velocity_x = Var(m.N, m.signs, bounds = (0.0,None)) # the sign will account for -ve velocities
lamb_foot_velocity_x = sym.lambdify(sym_list,(foot_position.jacobian(q)*dq)[0].simplify(),modules = [func_map])
def def_foot_velocity(m,n):
    #if n == 1: return Constraint.Skip
    var_list = get_var_list_zeros(m,n)
    return m.foot_velocity_x[n,'+ve']-m.foot_velocity_x[n,'-ve'] == lamb_foot_velocity_x(*var_list)
m.foot_velocity_constraint = Constraint(m.N, rule = def_foot_velocity)

m.GRF_y = Var(m.N, bounds = (0.0,10*total_mass*m.g.value)) # ground reaction forces
m.GRF_x = Var(m.N, bounds = (0.0,10*total_mass*m.g.value)) # ground reaction forces
m.alpha = Var(m.N, bounds = (0,1))
m.mu_s = Param(initialize = 1)
m.mu_d = Param(initialize = 0.6)
def def_Ft(m,n):
    #if n == 1: return Constraint.Skip
    dx_mag = m.foot_velocity_x[n,'+ve'] + m.foot_velocity_x[n,'-ve']
    mu = (m.mu_s - m.mu_d)/(1 + delta*dx_mag) + m.mu_d
    Ft_mag = mu*m.GRF_y[n]
    return m.GRF_x[n] == (2*m.alpha[n]-1)*Ft_mag
m.def_Ft = Constraint(m.N, rule = def_Ft)



# ground_constraints = ['contact','dx_exclusive','slip_+ve','slip_-ve'] 

m.ground_varA = Var(m.N, m.ground_constraints, bounds = (0.0, None))
m.ground_varB = Var(m.N, m.ground_constraints, bounds = (0.0, None))
m.ground_penalty = Var(m.N, m.ground_constraints, bounds = (0.0,10.0)) # penalty

def def_ground_varA(m,n,gc):
    #if n == 1: return Constraint.Skip
    if gc == 'contact':
        if n < N:
            return m.ground_varA[n,gc] == m.foot_position_y[n+1]#+m.foot_position_y[n+1]
        else:
            return m.ground_varA[n,gc] == m.foot_position_y[n]
    if gc == 'dx_exclusive':
        return m.ground_varA[n,gc] == m.foot_velocity_x[n,'+ve']
    if gc == 'slip_+ve':
        return m.ground_varA[n,gc] == m.foot_velocity_x[n,'+ve']
    if gc == 'slip_-ve':
        return m.ground_varA[n,gc] == m.foot_velocity_x[n,'-ve']
m.def_ground_varA = Constraint(m.N, m.ground_constraints, rule = def_ground_varA)

def def_ground_varB(m,n,gc):
    #if n == 1: return Constraint.Skip
    if gc == 'contact':
        return m.ground_varB[n,gc] == m.GRF_y[n]
    if gc == 'dx_exclusive':
        return m.ground_varB[n,gc] == m.foot_velocity_x[n,'-ve']
    if gc == 'slip_+ve':
        return m.ground_varB[n,gc] == m.alpha[n]
    if gc == 'slip_-ve':
        return m.ground_varB[n,gc] == (1-m.alpha[n])
m.def_ground_varB = Constraint(m.N, m.ground_constraints, rule = def_ground_varB)

# complementarity
def ground_complementarity(m,n,gc):
    #if n == 1: return Constraint.Skip
    return  m.ground_varA[n,gc]*m.ground_varB[n,gc] <= m.ground_penalty[n,gc]
m.ground_complementarity = Constraint(m.N, m.ground_constraints, rule = ground_complementarity)

m.GRF_y[N].fix(0.0)
m.GRF_x[N].fix(0.0)


ERROR: Rule failed when generating expression for constraint def_Ft with index
    1: NameError: name 'delta' is not defined
ERROR: Constructing component 'def_Ft' from data=None failed: NameError: name
    'delta' is not defined


NameError: name 'delta' is not defined

In [ ]:
for dof in DOFs:
    m.dq[1,dof].fix(0.0)
    
# Set graph limits 
x_low_lim = -1.0
x_high_lim = 1.0
y_low_lim = -0.5
y_high_lim = 2.0

m.F_ax = Var(m.N)
tmp = 0.0
for n in range (1,N+1):
    if test == 'HDT':
        m.F_ax[n].fix(0.0)
    else:
        if n%10 == 0:
            m.F_ax[n].fix((100.0/N)*n)
            tmp = (100.0/N)*n
        else:
            m.F_ax[n].fix(tmp)

<a id='initial_conditions'></a>

In [ ]:
if test == "HDT":
    #  ----------------------------- HIGH DROP TEST ---------------------------

    # initial condition
    m.q[1,'x_body'].fix(0.0)
    m.q[1,'y_body'].fix(1.0)
        
else:
    m.q[1,'x_body'].fix(0.0)
    m.q[1,'y_body'].fix(0.0)

In [ ]:
# COST FUNCTION -------------------------------------------------------------------------------------------------------------
def CostFun(m):
    row = 1e0 # scale the penalty
    actuation_sum = 0.0
    pen_sum = 0.0
    
    for n in range(1,N+1):
        actuation_sum += (m.F_ax[n])**2
        for gc in ground_constraints:
            pen_sum += m.ground_penalty[n,gc] 
            
    return row*pen_sum # actuation_sum
        
m.Cost = Objective(rule = CostFun)

In [ ]:
''' Turning EOM back to pyomo (from sympy) '''
#create the constraints
def EOM_x_body(m,n):
    var_list = get_var_list_values(m,n)
    return lamb_EOMs['x_body'](*var_list) == 0
m.EOM_x_body_constraint = Constraint(m.N, rule = EOM_x_body)

def EOM_y_body(m,n):
    var_list = get_var_list_values(m,n)
    return lamb_EOMs['y_body'](*var_list) == 0
m.EOM_y_body_constraint = Constraint(m.N, rule = EOM_y_body)

# def dynamics(m,n,dof):
#     if n == 1: return Constraint.Skip    
#     var_list = get_var_list_values(m,n)
#     return lamb_EOMs[dof](*var_list) == 0
# m.dynamics = Constraint(m.N, m.DOFs, rule = dynamics)

In [ ]:
# solving
opt = SolverFactory('ipopt',executable = '/home/uct/Trajectory_Optimization/IPOPT/build/bin/ipopt')
#opt = SolverFactory('ipopt',executable = '/home/zubair/build/bin/ipopt')
opt.options["linear_solver"] = 'ma86'

# solver options
opt.options["expect_infeasible_problem"] = 'yes'
#pt.options["linear_system_scaling"] = 'none'
#opt.options["mu_strategy"] = "adaptive"
opt.options["print_level"] = 0     # prints a log with each iteration (you want to this - it's the only way to see progress.)
opt.options["max_iter"] = 30000*10   # maximum number of iterations
opt.options["max_cpu_time"] = 600*10  # maximum cpu time in seconds
opt.options["Tol"] = 1e-6          # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.

results = opt.solve(m, tee = True)

# model.piston_complementarity.deactivate()
# result = default_solver('/usr/local/bin/ipopt', approximate_hessian=False).solve(model, tee=True)
# if result.solver.termination_condition != TerminationCondition.optimal: quit()
# # solve for discrete piston control
# model.piston_complementarity.activate()
# result = default_solver('/usr/local/bin/ipopt', approximate_hessian=False, warm_start=True).solve(model, tee=True)

In [ ]:
print(results.solver.termination_condition) # check if optimal

In [ ]:
#animate it
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

# fig1, ax1 = plt.subplots(1,1) #create axes
# ax1.set_aspect('equal')

# # arrays to store values
# N_time = []

# def plot_robot(i,m,ax): #update function for animation
#     ax.clear()
#     ax.set_xlim([x_low_lim,x_high_lim+3.0])
#     ax.set_ylim([y_low_lim,y_high_lim])
#     N_time.append(i)
     
#     plot_body = {}
    
#     # plot body
#     plot_body.update({('x_back'):m.q[i,'x_body'].value})
#     plot_body.update({('y_back'):m.q[i,'y_body'].value})
#     ax.plot([plot_body['x_back'],plot_body['x_back']],[plot_body['y_back']+0.1,plot_body['y_back']],color='xkcd:black',linestyle='solid',markersize = 3.0)
    
#     ax.plot([-10,10],[0,0]) # plot the ground
    
    
# update = lambda i: plot_robot(i,m,ax1) # lambdify update function

# animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=True)

# HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook


In [ ]:
# m.q[18,'x_body'].value

In [ ]:
mu_values = []
nodes = []
vel = []
for n in range(1,1+N):
    #print(m.F_ax[n].value)
    #print(m.ddq[n,'y_body'].value)
    nodes.append(n)
    dx_mag = m.foot_velocity_x[n,'+ve'].value + m.foot_velocity_x[n,'-ve'].value
    mu = (m.mu_s - m.mu_d)/(1 + delta*dx_mag) + m.mu_d
    mu_values.append((m.mu_s.value - m.mu_d.value)/(1 + delta*dx_mag) + m.mu_d.value)
    vel.append(m.dq[n,'x_body'].value)

plt.figure()
plt.scatter(vel,mu_values,1)
plt.title('delta = {0}'.format(delta))
plt.ylabel('mu')
plt.xlabel('x_velocity')

In [ ]:
for vel_i in vel:
    if vel_i<(0.1+hm*10) and vel_i>1e-7:
        print(vel_i)

In [ ]:
def max_grad(vel,mu_values):
    grad = []
    
    for i in range (1,len(vel)-1):
        grad.append((mu_values[i+1]-mu_values[i])/(vel[i+1]-vel[i]))
        
    index_value = grad.index(min(grad))
    print('gradient = ',min(grad))
    print('Vth = ', vel[2+index_value])
    #return 
            
max_grad(vel,mu_values)